In [ ]:
# | default_exp _cli

In [ ]:
# | export

import subprocess
from pathlib import Path
from typing import *

import typer

In [ ]:
from typer.testing import CliRunner

In [ ]:
runner = CliRunner()

In [ ]:
# | export

def _filter_rc_branches(all_branches: str) -> str:
    return "\n".join(
        [
            i.split(" ")[0]
            for i in all_branches.split("\n")
            if i != "" and not i.split(".")[-1].split(" ")[0].isdigit()
        ]
    )

In [ ]:
all_branches="0.0.5rc10 [dev]\n0.0.5dev\n0.0.5rc0\n0.0.4 [latest]\n0.0.3\n0.0.2\n0.0.1\n"
expected='0.0.5rc10\n0.0.5dev\n0.0.5rc0'
actual = _filter_rc_branches(all_branches)
print(actual)
assert actual == expected

0.0.5rc10
0.0.5dev
0.0.5rc0


In [ ]:
all_branches="0.0.4 [latest]\n0.0.3\n0.0.2\n0.0.1\n"
expected=''
actual = _filter_rc_branches(all_branches)
print(actual)
assert actual == expected

In [ ]:
# | export

def _get_version_numbers(all_versions: str) -> str:
    return [v.split(" ")[0] for v in all_versions.split("\n")]

In [ ]:
all_versions='0.0.5rc10 [dev]\n0.0.5dev\n0.0.5rc0'
expected = ['0.0.5rc10', '0.0.5dev', '0.0.5rc0']

actual = _get_version_numbers(all_versions)
print(actual)
assert actual == expected

['0.0.5rc10', '0.0.5dev', '0.0.5rc0']


In [ ]:
# | export

def _filter_exclustion_list(versions_to_delete: List[str], exclude: str) -> List[str]:
    exclusion_list = [i.strip().split(" ")[0] for i in exclude.split(",")]
    return list(set(versions_to_delete) - set(exclusion_list))

In [ ]:
versions_to_delete = ['0.0.5rc1', '0.0.5rc0']
exclude = "0.0.5rc1 [dev], 0.0.5rc0 [old-dev]"
expected = []

actual = _filter_exclustion_list(versions_to_delete, exclude)
print(actual)
assert actual == expected

versions_to_delete = ['0.0.5rc1', '0.0.5rc0', '0.0.1rc0']
exclude = "0.0.5rc1 [dev], 0.0.5rc0 [old-dev]"
expected = ['0.0.1rc0']

actual = _filter_exclustion_list(versions_to_delete, exclude)
print(actual)
assert actual == expected

[]
['0.0.1rc0']


In [ ]:
# | export

def _add_git_args(cmd: list[str], rebase: bool, ignore: bool, prefix: str) -> List[str]:
    if rebase: 
        cmd = cmd + ["--rebase"]
    if ignore:
        cmd = cmd + ["--ignore"]
    if prefix:
        cmd = cmd + ["--prefix", prefix]
        
    return cmd
    

In [ ]:
cmd=["-b", "branch"]
rebase = True
ignore = False
prefix = "path"
expected = ["-b", "branch", "--rebase", "--prefix", "path"]

actual = _add_git_args(cmd, rebase, ignore, prefix)
print(actual)
assert actual == expected

cmd=["-b", "branch"]
rebase = False
ignore = True
prefix = "path"
expected = ["-b", "branch", "--ignore", "--prefix", "path"]

actual = _add_git_args(cmd, rebase, ignore, prefix)
print(actual)
assert actual == expected

cmd=["-b", "branch"]
rebase = True
ignore = True
prefix = None
expected = ["-b", "branch", "--rebase", "--ignore"]

actual = _add_git_args(cmd, rebase, ignore, prefix)
print(actual)
assert actual == expected

['-b', 'branch', '--rebase', '--prefix', 'path']
['-b', 'branch', '--ignore', '--prefix', 'path']
['-b', 'branch', '--rebase', '--ignore']


In [ ]:
# | export

_app = typer.Typer(help="")

@_app.command(
    help="Delete the deployed release candidate documentations.",
)
def delete_rc_docs(
    branch: str = typer.Option(
            "gh-pages",
            "--branch",
            "-b",
            help='The branch name to commit to',
        ),
    remote: str = typer.Option(
            "origin",
            "--remote",
            "-r",
            help='The origin name to push to',
        ),
    config_file: str = typer.Option(
            "mkdocs/mkdocs.yml",
            "--config-file",
            "-F",
            help='The MkDocs configuration file to use',
        ),
    rebase: bool = typer.Option(
            True,
            help='Rebase with remote brancj',
        ),
    ignore: bool = typer.Option(
            False,
            help='Ignore remote status',
        ),
    prefix: str = typer.Option(
            None,
            "--prefix",
            help='subdirectory within {branch} where docs are located',
        ),
) -> None:
    """Delete the deployed release candidate documentations."""
    try:
        cmd = ['-b', branch, '-r', remote, '-F', config_file]
        cmd = _add_git_args(cmd, rebase, ignore, prefix)
        nl = "\n"
                
        result = subprocess.run(['mike', 'list'] + cmd, stdout=subprocess.PIPE)
        deployed_docs_list = result.stdout.decode('utf-8')
        deployed_docs_list = _filter_rc_branches(deployed_docs_list)
        
        if deployed_docs_list == "":
            typer.echo("No release candidate documentation versions have been deployed.")
            return
        
        typer.echo(f"List of deployed release candidate documentation versions: \n{deployed_docs_list}\n")        
        option = typer.prompt("\nSelect an option: \n1. Delete all versions listed above \n2. Retain some versions while deleting others \n3. Abort\n\n")
        
        if option == "3":
            typer.echo("Aborted.")
            return
        
        versions_to_delete = _get_version_numbers(deployed_docs_list)
        if option == "2":
            while True:
                exclude = typer.prompt("Enter the names of the versions to retain, separated by comma (eg: 1.0.0rc0, 1.0.0rc1)")
                versions_to_delete = _filter_exclustion_list(_get_version_numbers(deployed_docs_list), exclude)
                typer.echo(f'Modified list of deployed release candidate documentation versions:\n{nl.join(versions_to_delete)}')
                delete = typer.confirm("Delete the above version(s)?")
                if delete:
                    break
            
        if len(versions_to_delete) == 0:
            typer.echo("No release candidate documentation version found to delete.")
        else:
            cmd = versions_to_delete + ['-b', branch, '-r', remote, '-F', config_file, '--push', '--force']
            cmd = _add_git_args(cmd, rebase, ignore, prefix)

            result = subprocess.run(['mike', 'delete'] + cmd, stdout=subprocess.PIPE)
            if result.returncode == 0:
                typer.echo(f"\nSuccessfully deleted the below version(s):\n{nl.join(versions_to_delete)}")
        
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
result = runner.invoke(_app, input="3")
print(result.stdout)
assert result.exit_code == 0, result.exit_code

List of deployed release candidate documentation versions: 
0.0.5rc1
0.0.5rc0


Select an option: 
1. Delete all versions listed above 
2. Retain some versions while deleting others 
3. Abort

: 3
Aborted.

